In [2]:
import numpy as np
import matplotlib as plt
import subprocess

from Bio.PDB import PDBParser
from Bio.PDB import Superimposer

### Make PDB Files

In [1]:
# Rename the chain name
# Define the input and output file paths

# f_chain = "500-30"
# f_chain = "mcp-2000-20"
# input_file = f"/u01/project/chenwei-embuild/AlignOT/6n52/6n52-{f_chain}-0.pdb"
# output_file = f"{input_file}_rechain.pdb"

input_file = "/u01/project/chenwei-embuild/AlignOT/6n52/6n52-2000-l1.pdb"
output_file = f"{input_file}_rechain.pdb"

# Open the input and output files
with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
    # Iterate through each line in the input file
    for line in infile:
        # Check if the line starts with "ATOM" and has a chain identifier "A"
        if line.startswith("ATOM") and line[21] == "A" or line.startswith("TER") and line[21] == "A":
            # Replace the chain identifier "A" with "B"
            modified_line = line[:21] + "B" + line[22:]
            # Write the modified line to the output file
            outfile.write(modified_line)
        else:
            # If not an ATOM line or doesn't have chain "A", write as is
            outfile.write(line)

In [16]:
# Concatenate two files

import subprocess
# Define the terminal command
# command = f"cat ./AlignOT/6n52/6n52-{f_chain}-1.pdb {output_file} > ./AlignOT/6n52/6n52OT-{f_chain}.pdb"
command = f"cat ./AlignOT/6n52/6n52-3000-right1.pdb ./AlignOT/6n52/6n52-3000-left2.pdb_rechain.pdb > ./AlignOT/6n52/6n52_l2r1.pdb"

# Run the command
subprocess.run(command, shell=True)

CompletedProcess(args='cat ./AlignOT/6n52/6n52-3000-right1.pdb ./AlignOT/6n52/6n52-3000-left2.pdb_rechain.pdb > ./AlignOT/6n52/6n52_l2r1.pdb', returncode=0)

In [17]:
# Remove the "END" except the last one

def remove_first_end(input_file, output_file):
    with open(input_file, 'r') as infile:
        lines = infile.readlines()

    with open(output_file, 'w') as outfile:
        end_found = False
        for line in lines:
            if not end_found and line.strip() == "END":
                end_found = True
                continue
            outfile.write(line)

# Usage
input_file = f'./AlignOT/6n52/6n52OT-{f_chain}.pdb'
output_file = input_file

remove_first_end(input_file, output_file)

In [18]:
# Delete unusefull files
command = f"rm ./AlignOT/6n52/6n52-{f_chain}-0.pdb ./AlignOT/6n52/6n52-{f_chain}-0_rechain.pdb ./AlignOT/6n52/6n52-{f_chain}-1.pdb"

# Run the command
subprocess.run(command, shell=True)

CompletedProcess(args='rm ./AlignOT/6n52/6n52-500-30-0.pdb ./AlignOT/6n52/6n52-500-30-0_rechain.pdb ./AlignOT/6n52/6n52-500-30-1.pdb', returncode=0)

### Load PDB Structures


In [4]:
# # pdb name
# pdb_name = "6n52"
# custom_name = "m_mcp"

# Load the reference and predicted structures
# ref_path = f"/u01/project/chenwei-empot/5t5h.cif"
# pred_path = f"/u01/project/chenwei-embuild/embuild_docker/outputs/{pdb_name}"

reference_pdb = f"/u01/project/chenwei-empot/5t5h.cif"
# paper_pdb = f'{ref_path}/{pdb_name}_paper.pdb'
# af_pdb = f"/u01/project/chenwei-embuild/embuild_docker/pdbs/6n52/6n52_h/6N52_h.pdb"
# fft_pdb = f"/u01/project/chenwei-embuild/AlignOT/{pdb_name}/{pdb_name}_FFT.pdb" 

predicted_pdb = f'"/u01/project/chenwei-empot/5t5h.cif"'


print("Ref: ", reference_pdb)
# print("FFT: ", fft_pdb)
print("Predicted: ", predicted_pdb)

# print("Alphafold: ", af_pdb)

Ref:  /u01/project/chenwei-empot/5t5h.cif
Predicted:  "/u01/project/chenwei-empot/5t5h.cif"


### Calculate RMSD & TM-score


In [8]:
MMalign = '/u01/project/chenwei-empot/MMalign/MMalign'

# # Define the command to run
# command = [MMalign, reference_pdb, paper_pdb]
# command = [MMalign, reference_pdb, fft_pdb]
command = [MMalign, reference_pdb, predicted_pdb]


# Run the command
try:
    result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True, check=True)
    # Access the standard output of the command
    output = result.stdout
    # Print or process the output as needed
    print("MMalign output:", output)
except subprocess.CalledProcessError as e:
    # Handle errors if the command fails
    print("Error:", e.stderr)

Error: Warning! Cannot parse file: "/u01/project/chenwei-empot/5t5h.cif". Chain number 0.



In [8]:
# Extract RMSD and TM-score
rmsd_index = output.index("RMSD=") + 6
rmsd_end_index = output.index(",", rmsd_index)
rmsd = float(output[rmsd_index:rmsd_end_index].strip())

tm_score_index = output.index("TM-score=") + 9
tm_score_end_index = output.index("(", tm_score_index)
tm_score = float(output[tm_score_index:tm_score_end_index].strip())

print("RMSD:", rmsd)
print("TM-score:", tm_score)

RMSD: 7.6
TM-score: 0.71805


In [ ]:
## FFT (single chain fitting):
# RMSD: 6.37
# TM-score: 0.68454

In [ ]:
## paper ##
# RMSD: 1.38
# TM-score: 0.98859

## mc_mcp: ##
# RMSD: 1.36
# TM-score: 0.98903

## mc_tighttarget-mcp vs ref ##
# RMSD: 1.4
# TM-score: 0.98829

## mc_widetarget-mcp vs ref ##
# RMSD: 1.42 
# TM-score:  0.98806

## mc_highres-mcp vs ref ##
# RMSD: 1.38
# TM-score:  0.98867

## hc_mcp: ##
# RMSD: 1.44
# TM-score: 0.98828

## m_mcp: ##
# RMSD: 2.4
# TM-score: 0.96571

## h_mcp: ##
# RMSD: 4.11
# TM-score: 0.90955


## monomer_cut vs homomer_cut ##
# RMSD: 1.54
# TM-score: 0.98913

## monomer vs homomer ##
# RMSD: 3.98
# TM-score: 0.9106


In [7]:
from spyrmsd import io, rmsd

In [ ]:
# Load reference and predicted structures as RDKit molecules
ref = io.load_molecule(reference_pdb)
pred = io.rdkit.load_molecule(predicted_pdb)

# Perform sequence alignment using Biopython
ref_aligned, pred_aligned = spyrmsd.align.sequence_align(ref, pred)

# Calculate RMSD using Kabsch algorithm
rmsd = spyrmsd.rmsd.kabsch_rmsd(ref_aligned.GetConformer(), pred_aligned.GetConformer())

# Print RMSD
print(f"RMSD = {rmsd:.3f} Å")


In [ ]:
reference_pdb, predicted_pdb